## 序列模型

### 自回归模型

In [ ]:
import sys
sys.path.append('..')
import torch
from torch import nn
import d2l

In [ ]:
T = 1000 # 总共产生1000个点
time = torch.arange(1, T + 1, dtype=torch.float32)
x = torch.sin(0.01 * time) + torch.normal(0, 0.2, (T,))
d2l.plot(time, [x], 'time', 'x', xlim=[1, 1000], figsize=(6, 3))

In [ ]:
tau = 4
features = torch.zeros((T - tau, tau))
for i in range(tau):
  features[:, i] = x[i: T - tau + 1]
labels = x[tau:].reshape((-1, 1))